# Zawaansowane widoki część 1

## Agenda
1. Request & response
2. Do czego służą widoki ?
3. Rodzaje widoków
4. Django template language

### Request vs. response
Jest to podstawowy sposób komunikacji odbywającej się w protokole HTTP. Gdy chcemy uzyskać dostęp do jakiegoś miejsca np. `http://www.google.com` wysyłamy **żądanie** (**reqest**) do serwera, który hostuje ten adres. Ten serwer odbiera nasz **request**, przetwarza go i zwraca odpowiedni **response**.

W Django mamy wbudowane obiekty, które pozwalają nam obsługiwać zarówno żądania jak i odpowiedzi na te żądania.

**HttpRequest**
W Django za obsługę żądania HTTP odpowiada obiekt HttpRequest. Klasa definiująca ten obiekt znajduje się w module django.http.
Zawiera wszystkie pola nagłówków żądania. Te pola możesz wykorzystywać w swojej aplikacji.
Obiekt HttpRequest jest obsługiwany przez framework. W większości przypadków nie będziecie o nim nawet myśleć ;-)

**HttpResponse**
Za obsługę odpowiedzi w Django odpowiada obiekt HttpResponse. Klasa definiująca ten obiekt znajduje się w module django.http.

### Do czego służą widoki ?
Widok jest to kluczowy komponent frameworku, pisałem już o nich w pierwszym rozdziale. Są to funkcję / klasy które przyjmują obiekt **HttpRequest** przetwarzają go i zwracają obiekt **HttpResponse**. Każdy widok musi być przypisany do URL'a. Widoki są używane do rzeczy takich jak wyciąganie rzeczy z bazy, modyfikowanie danych w bazie, obsługa formularzy, zwracanie HTML'a ..

Wróćmy zatem do naszego projektu. Stworzyliśmy ostanio model `Article` oraz nauczyliśmy się jak uzyskiwać infromacje z tego modelu używając `querysetów`, wykorzystajmy tą wiedzę tworząc widok, dostępny pod adresem `/articles`, który będzie zwracał listę wszystkich artykułów.
W pliku `views.py` napiszmy:

```python
from django.http import HttpResponse
from django.shortcuts import render
from my_app.models import Article

...

def articles(request):
    articles = Article.objects.all()
    return render(request, "articles.html", {
        "articles": articles
    })
```
Stworzyliśmy widok **articles**, który przyjmuje obowiązkowy parametr **request**. W widoku wyciągamy z bazy wszystkie artykuły z modelu **Article**. Zwracamy response używając funkcji **render()** zaimportowanej z `django.shortcuts`, ta funkcja przyjmuje 3 parametry:
* obiekt **request**
* plik html w który ma być wyrenderowany. Domyślna lokalizacja w której Django szuka templatek to jest folder `templates` wewnątrz aplikacji. Dlatego plik **articles.htm** znajduje się w **my_app/templates/articles.html**
* zmienne, które zostaną przekazane do tego html'a. Zbiór tych zmiennych nazywamy **kontekstem** nazwy które piszemy w cudzysłowie to będą nazwy do których będziemy się odnosili w HTML'u

Stwórzmy więc plik **articles.html** w folderze **templates** wewnątrz aplikacji **my_app**.

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <title>Lista artykułów</title>
</head>
<body>
    <h1>Lista artykułów</h1>
    <hr>
    {% for article in articles %}
        <h5>{{ article.title }}</h5>
        <p>Author: <span>{{ article.author.username }}</span></p>
        <p> 
            Likes: <span style="font-weight: bold;">{{ article.likes }}</span> 
            Dislikes: <span style="font-weight: bold;">{{ article.dislikes }}</span>
        </p>
        <br>
        <br>
    {% endfor %}
</body>
</html>
```

Trochę dziwny ten HTML, prawda ;) ? Django posiada swój silnik zwany **Django Template Language** który pozwala pisać zwykły html z dodatkami takimi jak wyświetlanie zmiennych przekazanych w kontekscie, używanie różnych **tagów** takich jak pętla czy instrukcja warunkowa oraz wiele więcej.


Musimy przypisać nasz widok pod wybrany przez nas URL. Przejdźmy do pliku **urls.py** znajdującego się w folderze **new_project** i napisz:

```python
from django.contrib import admin
from django.urls import path

from my_app import views 

urlpatterns = [
    path('admin/', admin.site.urls),
    path("", views.home_view, name="home"),
    path("articles/", views.articles, name="articles"), # <-- ta linia została dodana
]
```


Uruchom serwer i przejdź na adres http://localhost:8000/articles
Powinien się wyświetlić następujący rezultat:
![alt image](https://i.imgur.com/ngL4MTe.png)


### Rodzaje widoków

Django posiada dwa rodzaje pisania widoków:
* **function-based views** (FBVs) - widoki funkcyjne
* **class-based views** (CBVs) - widoki klasowe

Na początku Django posiadało tylko widoki funkcyjne, potem dodali również widoki klasowe, żeby ułatwić pisanie często powtarzającej się logiki w naszych widokach (dodawanie danych do bazy, dodawanie formularza, lista elementów z bazy itp.). Zarówno widoki funkcyjne jak i klasowe mają swoje zalety i wady, dlatego nie możemy mówić, że należy używać tylko jednej metody podczas pisania naszych widoków.

#### Widoki funkcyjne
Są omawiane na początku nauki Django. Są to funkcje, które przyjmują **zawsze** jako pierwszy argument obiekt HttpRequest i ten widok zwraca HttpResponse, czyli np. zwraca HTML'a. Widok może również przyjmować argumenty dodatkowe argumenty oprócz pierwszego obowiązkowego argumenty **request**. Przykład:

```python

def article_detail(request, article_id):
    article = Article.objects.get(id=article_id)
    return render(request, "article_detail.html", {"article": article})
```

Jak widać na przykładzie widok przyjmuje argument article_id, żeby ten argument był rozpoznany w pliku urls.py napiszemy:

```python
urlpatterns = [
    path("article/<article_id>", article_detail, name="article_detail"),
]
```

Podczas rejestrowania widoki w pliku **urls.py** w ścieżce wewnątrz `<>` tworzymy placeholder na argument `article_id`. Teraz wchodząc na link `http://localhost:8000/article/1` widok zwróci nam obiekt `Article` o id = 1.

**Zalety**
* Łatwe w zaimplementowaniu
* Łatwość w czytaniu
* Możliwość użycia dekoratorów
* Pełna kontrola nad kodem
* Dobre do pisania logiki dla bardziej zawiłego zastosowania funkcji

**Wady**
* Ciężko reużywać kodu napisanego w danym widoku
* Zarządzanie metodami HTTP używając warunków `if else`

#### Widoki klasowe
Drugi sposób pisania widoków. Ich dużą zaletą jest generyczność, czyli został dla nas zachowany pewien sposób abstracji. Przykładowo mając taki widok funkcyjny:

```python

def some_view(request):
    if request.method == "GET":
        # do something
    elif request.method == "POST":
        # do something
    elif request.method == "PUT":
        # do something
```

Możemy sobie uprościć sprawę używając widoku klasowego:

```python

from django.views import View

class SomeView(View):
    
    def get(self, request):
        # do something
    
    def post(self, request):
        # do something
    
    def put(self, request):
        # do something
     
```
To pozwala nam się trzymać zasad pisania małych funkcji / metod, które są łatwe do czytania oraz do modyfikacji.

Jak pokazałem na powyższym przykładzie aby pisać widoki klasowe, nasza klasa musi dziedziczyć z **django.views.View** i następnie piszemy logikę w metodach nazwanych na podstawie metod protokołu HTTP (get, post, put, delete ...).

**Rejestrowanie widoku:**
W przypadku rejestrowaniu wiodoków klasowych istnieje minimalna różnica w porównaniu do widoków funkcyjnych. Zpobaczymy to na przykładzie. Załóżmy że chcemy zarejestrować widok **SomeView** pod adresem: **/home** Przejdźmy do pliku **urls.py** i napiszmy:

```python

from django.urls import path
from my_app import views

urlpatterns = [
    path("home/", views.SomeView.as_view(), name="home"), # <-- zwróć uwagę na as_view()
]
```

Jak widać również jako drugi argument w funkcji `path()` przekazujemy nazwę widoku, jednak musimy wywołać tę metodę używając metody **as_view()**

**Zalety**
* DRY - ograniczenie duplikacji kodu
* Rozszerzalność - CBVs mogą być rozszerzane za pomocą [Mixinów](https://docs.djangoproject.com/en/2.2/topics/class-based-views/mixins/)
* Struktura poprawiająca czytelność kodu
* Rozdzielenie logiki na odpowiednie metody protokołu HTTP
* wbudowane generyczne widoki klasowe

**Wady**
* Zaimplementowana za nas warstwa logiki - ciężko ją modyfikować
* Cieżko używa się dekoratorów


#### Podsumowanie
Decyzja o używaniu odpowiedniego rodzaju widków jest subiektywną oceną, jednak pomóc może nam ta grafika:


![alt_image](https://i.imgur.com/iSKe6JT.jpg)

### Django Template Language
Zapewne zdziwiła was składnia w pliku **article.html**. Pojawiła się jakaś pętla oraz dostęp do zmiennych. Django posiada swój silnik, który rozszerza normalną składnie HTMl'a o dodatkowe funkcjonalności takie jakie:
* **zmienne**
* **filtry**
* **tagi**


Omówmy zatem poszczególne pozycje z listy:

1. **Zmienne**

Pewnie pamiętasz, że trzecim parametrem funkcji **render()** był słownik w następującej postaci:
```python
{
    "variable1": 10,
    "variable2": 'String'
}

```
Dzięki temu przekazaliśmy do templatki zmienne **variable1** oraz **variable2**. Do zmiennych możemy odwoływać się w następujący sposób: **{{ variable }}** Zmienną może być również obiekt dlatego możemy za pomocą kropki odwoływać się do jego atrybutów: **{{ person.first_name }}**

2. **Filtry**
Możesz modyfikować sposob wyświetlania zmiennych za pomocą filtrów. Przykładowo **{{ name|lower }}** zmieni tekst znajdujący się w zmiennej name na małe litery. Niektóre filtry oprócz nazwy posiadają argumenty, które one przyjmują np. filtr **date** przyjmuje format daty jaki chcemy wyświetlić. Takie argumenty przekazujemy po dwukropku **{{ some_date|date:"D M Y" }}** 
Django posiada wiele wbudowanych filtrów, pełną liste możesz zobaczyć [tutaj](https://docs.djangoproject.com/en/2.2/ref/templates/builtins/#built-in-filter-reference).
Istnieje również możliwość pisania własnych filtrów.


3. **Tagi**
Tagi występują w następujący sposób **{% tag %}**. Taki są bardziej zawaansowane niż zmienne, niektóre kontrolują tesks, inne tworzą pętle. Bardzo często tagi należy zamknąć przykładowo: **{% tag %} Some content in here {% endtag %}**. Listę wszsyskich wbudowanych tagów znajdziesz [tutaj](https://docs.djangoproject.com/en/2.2/ref/templates/builtins/#built-in-tag-reference). Tak samo jak w przypadku filtów, można pisać własne tagi.

Przykład, który zawiera wszystkie omawiane wyżej elementy **Django Template Language**

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <title>Document</title>
</head>
<body>
    <h1>User list</h1>
    {% for user in users %}
        <p>username: {{ user.username }}</p>
        <p>date created: {{ user.created_date|date:"D d M Y" }}</p>
    {% endfor %}    
</body>
</html>
```

#### Dziedziczenie templatek

Dobrym pomysłem jest pisanie tzw. templatki bazowej dla naszego projektu, która posiada wszystkie elementy które się nie zmieniają niezależnie na jakiej podstronie jesteśmy. Takimi elementami są np. tag `head`, nawigacja na górze oraz stopka, które pozostają niezmienne. Jedynie kontent się zmienia.

Stwórzymy wiec templatkę bazową w pliku **base.html**
```html
<!DOCTYPE html>
<html lang="en">

<head>
    <title>
        {% block title %}
        {% endblock %}
    </title>
</head>

<body>
    <nav>
        <ul>
            <li>Home</li>
            <li>Blog</li>
            <li>About</li>
        </ul>
    </nav>

    {% block content %}
    {% endblock %}

    <footer>
        <p>This is footer of our site</p>
    </footer>
</body>

</html>
```

oraz plik **home.html** który dziedziczy po tej templatce bazowej

```html
{% extends 'base.html' %}

{% block title %}
Home Page
{% endblock %}

{% block content %}
    <h1>Home page</h1>
    <p>Lorem ipsum ......</p>
{% endblock %}
```

daj sobie trochę czasu, żeby przeanalizować ten kod.
Plik **base.html** będzie naszą templatką bazową, zatem zawiera on wszystkie elementy, które się nie zmieniają, w nazym przykładzie będzie to tag head, nawigacja oraz stopka. To co się zmienia to kontent znajdujący się pomiędzy nawigacją a stopką. Korzystamy więc z tagu **{% block %}** który jest takim placeholderem. W tym miejscu zostanie umieszczony kontent templatki dziedziczącej. Dodatkowo tworzymy **{% block title %}** żeby każda dziedzicząca templatka mogła mieć swój tytuł strony.

W pliku **home.html** mówimy, że chcemy odziedziczyć z templatki bazowej używając składni **{% extends 'base.html' %}**. Nie umieszczamy już tagu head, nawigacji, ani stopki ponieważ będzie to wzięte z templatki bazowej. Tworzymy bloki o takiej samej nazwie jak w templatce bazowej czyli **title** oraz **content** i wewnątrz tych bloków umieszczamy odpowiednie informacje.


#### Includowaniem templatek

Innym sposobem na podział kodu html, jest tworzenie tzw. komponentów, czyli krótkich fragmentów, które odpowiadają za jedną rzecz. Możemy sobie np. stworzyć komponent nawigacji, komponent stopki itp...

Stwórzmy więc plik **navbar.html** w którym umieścimy następujący kod:
```html
<nav>
    <ul>
        <li>Home</li>
        <li>Blog</li>
        <li>About</li>
    </ul>
</nav>
```

Stwórzmy również plik **footer.html**:
```html
<footer>
    <p>This is my footer</p>
</footer>
```

Stwórzmy w końcu plik **home.html**:
```html

<!DOCTYPE html>
<html lang="en">
<head>
    <title>Home</title>
</head>
<body>
    {% include 'navbar.html' %}

    <h1>Home page content</h1>
    <p>Lorem ipsum ..</p>

    {% include 'footer.html' %}
</body>
</html>
```

Miejsca gdzie umieszczamy tag **include** zostaną zastąpione przez kod odpowiednich komponentów. Takie pisanie frontendu jest bardzo powszechne w najbardziej znanych frameworkach javascriptowych takich jak **React**, **Angular** oraz **Vue.js**.

### Zadanie domowe

Jest to kontynuacja projektu my_project który był tworzony w zadaniu domowym w prezentacji nr. 5
1. Stwórz widok funkcyjny `articles()` który będzie zarejestrowany pod adresem **articles/** ten widok będzie wyświetlał listę artykułów z modelu **Articles**. Niech widok zwraca za pomocą funkcji **render()** html o nazwie `articles.html`
2. Stwórz plik **articles.html** który będzie wyświetlał listę artykułów, użyj pętli for uzywając tagu **{% for %}**
3. Na takiej samej zasadzie stwórz drugi widok `article_detail()`


### Materiały
* [Class based views vs. function based views](https://medium.com/@ksarthak4ever/django-class-based-views-vs-function-based-view-e74b47b2e41b)
* [Django filters & tags](https://docs.djangoproject.com/en/2.2/ref/templates/builtins/)
* [Pisanie własnych filtrów i tagów](https://docs.djangoproject.com/en/2.2/howto/custom-template-tags/)